<a href="https://colab.research.google.com/github/MuhamaadUsman/RAG-using-Lama2/blob/main/Rag_using_OpenAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install python-multipart

!pip install typing-extensions<4.6.0
!pip install kaleido
!pip install cohere
!pip install langchain
!pip install langchain_core.pydantic_v1
from langchain.prompts import ChatPromptTemplate
!pip install langchain openai weaviate-client
!pip install tiktoken


import os
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.pydantic_v1 import BaseModel, PrivateAttr
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

/bin/bash: line 1: 4.6.0: No such file or directory
ERROR: Could not find a version that satisfies the requirement langchain_core.pydantic_v1 (from versions: none)
ERROR: No matching distribution found for langchain_core.pydantic_v1


ImportError: ignored

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-Key_here"

In [ ]:
documents = []
for file in os.listdir('/content/Data'):
    if file.endswith('.pdf'):
        pdf_path = '/content/Data/' + file
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)



In [ ]:
len(chunked_documents)

3316

In [ ]:
client = weaviate.Client(
  embedded_options=EmbeddedOptions()
)
vectorstore = Weaviate.from_documents(
    client=client,
    documents=chunked_documents,
    embedding=OpenAIEmbeddings(openai_api_key=openai_api_key),
    by_text=False
)


Started /root/.cache/weaviate-embedded: process ID 4330


ImportError: ignored

In [ ]:
retriever = vectorstore.as_retriever()

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
# import pandas as pd

# file_path = '/content/Evaluation set .xlsx'
# data = pd.read_excel(file_path)

# # Function to invoke RAG model with length check
# def invoke_rag_chain(query):
#     try:
#         result = rag_chain.invoke(query)
#         return result
#     except Exception as e:
#         return f"Limit exceeded: {e}"

# data['RAG answer'] = data['Query'].apply(lambda query: invoke_rag_chain(query + "?"))

# modified_file_path = '/content/Evaluation set Rag Result.xlsx'
# data.to_excel(modified_file_path, index=False)


In [ ]:
# query = "What percentage of surveyed farmers relied on traditional methods for controlling livestock diseases?"
# rag_chain.invoke(query)

In [ ]:
import random
import gradio as gr

def alternatingly_agree(message, history):
    answer = rag_chain.invoke(message)
    print(answer)
    return answer
gr.ChatInterface(alternatingly_agree).launch()

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import CountVectorizer

# data = pd.read_excel('/content/Evaluation set Rag Result.xlsx')

# def calculate_cosine_similarity(text1, text2):
#     vectorizer = CountVectorizer().fit_transform([text1, text2])
#     vectors = vectorizer.toarray()
#     return cosine_similarity([vectors[0]], [vectors[1]])[0][0]

# data['Cosine similarity'] = data.apply(lambda row: calculate_cosine_similarity(row['Answer'], row['RAG answer']), axis=1)

# modified_file_path = '/content/Evaluation set Rag Result.xlsx'
# data.to_excel(modified_file_path, index=False)
# print('Done')

Done
